# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# !pip install pyarrow==2.0.0
# !pip install umap
# wait_for_me = True # dummy code that prevents jupyter from executing the following cells before this one has completed

In [2]:
import time
import os
import numpy as np
import pandas as pd
from scipy import stats
import umap.umap_ as umap

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.cluster import OPTICS

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.layouts import column, gridplot
from bokeh.models import ColumnDataSource, FactorRange, Label
from bokeh.palettes import Spectral6, Cividis, Category10, Category20, RdYlBu
from bokeh.transform import factor_cmap
output_notebook()

from tqdm.notebook import tqdm

from datetime import datetime
from functools import partial

from IPython.display import clear_output

Loading BokehJS ...

In [3]:
pd.set_option('plotting.backend', 'pandas_bokeh')

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [4]:
# load in the data
# csv files are cumbersome to wrok with - convert them to the parquet format so we can load them quicker

def load_data(path, name):
    """
    Load a csv file if a parquet file does not exist otherwise load the parquet file. 
    If the parquet file does not exist it will be created and saved in the current working directory.
    This speeds up loading times for the next run (by about 4x). 
    
    INPUT
    path - path to the csv file
    name - name to save the csv file to in parquet format
    
    OUTPUT
    df - pandas dataframe with object columns converted to string
    """
    if not os.path.exists(f'{name}.parquet'):
        print(f'{name} parquet not found - loading csv')
        df = pd.read_csv(path, sep=';')
        print(f'{name} csv loaded')
        for c in df.columns:
            if df[c].dtype == object:
                df[c] = df[c].astype(str)
                print(f"{name}: {c} converted to string")
        df.to_parquet(f'{name}.parquet')
        print(f'{name} converted to parquet')
    else:
        print(f'{name} parquet found - loading')
        df = pd.read_parquet(f'{name}.parquet')
        print(f'{name} parquet loaded')
    return df

In [5]:
%%time        
# load the two datasets
azdias = load_data('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', 'azdias')
customers = load_data('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', 'customers')

azdias parquet found - loading
azdias parquet loaded
customers parquet found - loading
customers parquet loaded
CPU times: user 6.67 s, sys: 2.25 s, total: 8.92 s
Wall time: 1.86 s


In [6]:
# parse attributes sheet
attributes = pd.read_excel('DIAS Attributes - Values 2017.xlsx', header=1).drop(columns=['Unnamed: 0', 'Description']).fillna(method='ffill')
# make the attribute labels match the labels in the dataset where we could figure out what matches
attributes.loc[:,'Attribute'] = attributes['Attribute'].str.replace('_RZ', '')
attributes['Attribute'].replace(
    {
        'D19_BUCH': 'D19_BUCH_CD',
        'CAMEO_DEUINTL_2015': 'CAMEO_INTL_2015',
        'D19_KK_KUNDENTYP': 'KK_KUNDENTYP',
        'KBA13_CCM_1400_2500': 'KBA13_CCM_1401_2500',
        'SOHO_FLAG': 'SOHO_KZ',
    },
    inplace=True
)

# get number of values associated with an attribute 
nr_attribute_values = attributes.groupby('Attribute').count()['Value']
# filter out attributes that are only 1 row (these dont have a pre-devined value for unknowns)
attributes_filt = attributes[attributes['Attribute'].isin(nr_attribute_values[nr_attribute_values >  1].index)]

# Clean dataset meta info

In [7]:
def setup_dataframe(df, index_col='LNR'):
    """
    Setup the dataframe index to the specified column. For this data its the LNR column. 
    Rid the dataframe of columns that contain meta information on the data, that is not important to the task.
    The function runs in place and does not return a value.
    
    Input
    - DataFrame - dataframe to set the index on.
    - index_column - the column to use as the index.
    """
    # Entry ID = LNR (does not overlap between azdias and customers)
    df.set_index(index_col, inplace=True)
    # EINGEFUEGT tanslates to inserted and is in the form of a date
    # drop so we don't errenously correlate when the data appeared in the dataset to things we observe in the data
    df.drop(columns=['EINGEFUEGT_AM'], inplace=True) 

In [8]:
# setup both the datasets to prepare them for further analysis
setup_dataframe(azdias)
setup_dataframe(customers)

# Basic checks on number of columns

In [9]:
# check how many columns there are
print(
 f"{len(azdias.columns)} - number of columns in azdias dataframe\n" + 
 f"{len(customers.columns)} - number of columns in customers dataframe\n" +
 f"{len(set(azdias.columns).intersection(set(customers.columns)))} - number of columns that overlap betwen azdias and customers\n" +
 f"{len(nr_attribute_values)} - number of columns described in attributes file\n" +
 f"{len(set(nr_attribute_values.index) - set(azdias.columns))} - number of attribures not in dataframe columns\n" +
 f"{len(set(azdias.columns) - set(nr_attribute_values.index))} - number of columns in dataframe not in attributes" 
)
# 364 columns are too many to analyze individually. 
# All comuns in azdias is in customers.
# The attributes file does not contain information of all columns in the data.
# There are columns in the data not contained in the attributes file. These will need further investigation. 

364 - number of columns in azdias dataframe
367 - number of columns in customers dataframe
364 - number of columns that overlap betwen azdias and customers
314 - number of columns described in attributes file
4 - number of attribures not in dataframe columns
54 - number of columns in dataframe not in attributes


# Drop columns we know too little about
Columns we could manually treat or decode have been treated above so these are the remaining cases.

Often the speed with which we can get to a good enough answer is preferred for decision makers, so they can start working on their approach while futher analsysis refines the results. 

We can always come back later and try to squeeze more information out of these columns if required, if the rest of the dataset does not provide sufficient predictive capability given the bussiness objectives or for a version 2 iteration to improve. 

In [10]:
# identify columns in our data that is not described by the accompanying information sheet
low_to_no_info_columns = set(azdias.columns) - set(nr_attribute_values.index)
# drop the columns we know too little about
azdias.drop(columns=low_to_no_info_columns, inplace=True)
customers.drop(columns=low_to_no_info_columns, inplace=True)

# Check for duplicate rows

In [11]:
# find rows that are duplicated in our dataset
a_dups, c_dups = azdias.duplicated(), customers.duplicated()

In [12]:
# Check the number of missing values in this duplicate data - turns out its mostly missing anway
(azdias.loc[a_dups].isna().sum()/a_dups.sum()).sort_values(ascending=False).head(210)

KK_KUNDENTYP                   1.000000
D19_GESAMT_ONLINE_QUOTE_12     0.999885
D19_VERSAND_ONLINE_QUOTE_12    0.999885
D19_LOTTO                      0.999885
D19_BANKEN_ONLINE_QUOTE_12     0.999885
                                 ...   
ALTER_HH                       0.995736
WOHNDAUER_2008                 0.995736
SOHO_KZ                        0.995736
HH_EINKOMMEN_SCORE             0.001128
SEMIO_TRADV                    0.000000
Length: 210, dtype: float64

In [13]:
# There is some mismatch in the degree to which data is missing between the two datasets, nothing to worry about too much.
a_dups.sum()/len(azdias), c_dups.sum()/len(customers)

(0.058686902575231056, 0.2160791434474986)

In [14]:
# drop the duplicated rows
azdias.drop(index=(a_dups[a_dups==True].index), inplace=True)
customers.drop(index=(c_dups[c_dups==True].index), inplace=True)

# Basic data type cleaning

In [15]:
# instead of searching for object, remove known accepted datatypes
# there may be some strange things we could miss if we only look for object data types
cols_requiring_transformation = azdias.dtypes[(azdias.dtypes != np.int64) & (azdias.dtypes != np.float64)]
cols_requiring_transformation

CAMEO_DEU_2015     object
CAMEO_DEUG_2015    object
CAMEO_INTL_2015    object
OST_WEST_KZ        object
dtype: object

In [16]:
# X / XX missing data labels are very rare and goes up in frequency in both datasets as nans go up 
# conclusion: replace them with NaNs. 
pd.DataFrame([
    azdias[cols_requiring_transformation.index].isin(['X', 'XX']).sum()/len(azdias),
    azdias[cols_requiring_transformation.index].isin(['nan']).sum()/len(azdias),
    customers[cols_requiring_transformation.index].isin(['X', 'XX']).sum()/len(customers),
    customers[cols_requiring_transformation.index].isin(['nan']).sum()/len(customers),
], index=['A X/XX', 'A nan', 'C X/XX', 'C nan']).T

,A X/XX,A nan,C X/XX,C nan
CAMEO_DEU_2015,0.000445,0.055828,0.000839,0.061116
CAMEO_DEUG_2015,0.000445,0.055828,0.000839,0.061116
CAMEO_INTL_2015,0.000445,0.055828,0.000839,0.061116
OST_WEST_KZ,0.000000,0.048883,0.000000,0.057801


In [17]:
def process_column_dtype(val, base_type=float, nans=[]):
    """
    Convert value to type of base_type, unless str.lower(value) is in nans (all lower) then replace with np.NaN
    
    Input
    val - value to convert
    base_type - data type value is to be converted to
    nans = list of values that should be processed as a NaN
    
    Output
    value of type base_type or np.NaN
    """
    if str.lower(val) in nans:
        return np.NaN
    else:
        return base_type(val)
    
def datm(x):
    """
    Helper function to convert datetimes.
    
    Input - string in format %Y-%m-%d %H:%M:%S (e.g. 1992-02-10 14:13:59)
    Output - datetime 
    """
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

# create functions that can be used in a pandas apply (only 1 input argument)
# sometimes it's hard when you try to be DRY
process_column_cat = partial(process_column_dtype, base_type=str, nans=['nan', 'x', 'xx'])
process_column_datetime = partial(process_column_dtype, base_type=datm, nans=['nan', 'x', 'xx'])
process_column_float = partial(process_column_dtype, base_type=float, nans=['nan', 'x', 'xx'])

In [18]:
def apply_cleaning_process(df, col_to_proc):
    """
    Takes a dataframe and a dict mapping columns to the cleaning process to be applied.
    
    Input
    df - DataFrame to apply cleaning process to
    col_to_proc - dict with keys as the column names, and values as the process to apply to those columns
    
    Output
    None - modifies dataframe in place
    """
    for c, p in col_to_proc.items():
        df[c] = df[c].apply(p)

col_to_proc = {
    # columns that are numeric
    'CAMEO_DEUG_2015' : process_column_float,
    'CAMEO_INTL_2015' : process_column_float,
    # columns that are categories 
    'CAMEO_DEU_2015'  : process_column_cat,
    'OST_WEST_KZ'     : process_column_cat,
}

# clean both datasets
apply_cleaning_process(azdias, col_to_proc)
apply_cleaning_process(customers, col_to_proc)

In [19]:
def generate_categorical_replacement_dict(values):
    """
    Generates a dict that maps the values to a numeric.
    
    Input - list like of values
    Output - a dict with keys as the values provided, and a numeric value as the value. 
    """
    replacements = dict()
    for n, k in enumerate(values):
        if type(k) == str:
            replacements[k] = n+1
    return replacements

def categorical_to_numeric(df, col, pre_calculated_replacement=None):
    """
    Replaces non numeric categorical values with numeric values.
    Uses the sort order of the non numerics to assign a numeric value.
    Returns the replacement value mapping if one is not provided so the same mapping can be applied to other 
    dataframes columns ensuring consistency.
    
    Input
    df - A pandas dataframe
    col - A column that this transformation should be applied to
    pre_calculated_replacement - a dict that maps between the current column values and the new column values
    
    Returns 
    None if replacement is provided otherwise:
    replacement - a dict that maps between the current column values and the new column values
    """
    if pre_calculated_replacement is None:
        vals = df[col].sort_values().unique() # String categories with missing data. 
        replacement = generate_categorical_replacement_dict(vals)
    else:
        replacement = pre_calculated_replacement
    df[col].replace(replacement, inplace=True)
    if pre_calculated_replacement is None:
        return replacement
    else:
        return None

In [20]:
# convert these from categorial to numeric making sure the conversion is consistent between the datasets
CAMEO_DEU_2015_replacements = categorical_to_numeric(azdias, 'CAMEO_DEU_2015')
categorical_to_numeric(customers, 'CAMEO_DEU_2015', CAMEO_DEU_2015_replacements)
OST_WEST_KZ_replacements = categorical_to_numeric(azdias, 'OST_WEST_KZ')
categorical_to_numeric(customers, 'OST_WEST_KZ', OST_WEST_KZ_replacements)

In [21]:
# double check we have all object columns treated
azdias.dtypes[(azdias.dtypes != np.int64) & (azdias.dtypes != np.float64)]

Series([], dtype: object)

# Clean missing data

In [22]:
# most rows contain some form of missing data!
# dropping all rows with missing data would be fatal. 
(azdias.isnull().sum(axis=1) > 0).sum()/len(azdias), (customers.isnull().sum(axis=1) > 0).sum()/len(customers)

(0.6865045213000556, 0.5387380191693291)

In [23]:
def find_value_for_unknown(group):
    """
    Takes a dataframe of Attributes Values and Meanings. 
    The meanings are parsed to find the Values that represent unkown or missing data.
    If no unkown label exist, the minimum value in the group minus 1 is returned.
    
    Input - A dataframe (or dataframe group prodiced as part of a group by).
    Output - a list of values that should be used to label unknowns within the dataframe.
    """
    # all the labels indicating missing data
    missing_meanings = ['unknown', 'unknown / no main age detectable', 'no transactions known', 'no transaction known']
    # find all rows that contain a missing label or equivalent
    missing_values = group[group['Meaning'].isin(missing_meanings)]['Value'].values
    # if there is only 1 missing value check if it is a string
    # sometimes there is more than 1 missing Value but its formatted as a string after reading the spreadsheet
    if len(missing_values) == 1:
        if type(missing_values[0]) == str:
            # split the string and return all missing values as a list of ints
            return [int(i) for i in missing_values[0].split(',')]
    elif len(missing_values) > 1:
        # There should not be more than 1 row with a missing label - should not happen so throw an error
        raise ValueError('More than 1 row contains a missing definition.')
    else:
        pass
    return missing_values

missing_labels = attributes_filt.groupby('Attribute').apply(find_value_for_unknown)

In [24]:
# Calculate for each column in each dataset what % of the labels are the missing / unkown label
azdias_missing = dict()
customers_missing = dict()
for c in tqdm(missing_labels.index):
    if c in azdias.columns:
        azdias_missing[c] = azdias[c].isin(missing_labels.loc[c]).sum()/len(azdias)
    if c in customers.columns:
        customers_missing[c] = customers[c].isin(missing_labels.loc[c]).sum()/len(customers)

  0%|          | 0/307 [00:00<?, ?it/s]

In [25]:
# combine the missing label data with the NAN / NULL % into a single dataframe (this will make plotting easier)
all_data = pd.DataFrame(
    [azdias.isnull().sum()/len(azdias), 
     customers.isnull().sum()/len(customers),
     pd.Series(azdias_missing),
     pd.Series(customers_missing)], 
    index=['azdias_null', 'customers_null', 'azdias_missing', 'customers_missing']
).T
all_data.index.name='features'
all_data.fillna(0, inplace=True)

all_data['azdias_all'] = all_data['azdias_null'] + all_data['azdias_missing']
all_data['customers_all'] = all_data['customers_null'] + all_data['customers_missing']

all_data.sort_values(by='azdias_all', ascending=False, inplace=True)
all_data.head(10)

,azdias_null,customers_null,azdias_missing,customers_missing,azdias_all,customers_all
features,,,,,,
TITEL_KZ,0.025532,0.035643,0.971894,0.948975,0.997425,0.984618
D19_TELKO_ONLINE_DATUM,0.000000,0.000000,0.990222,0.985803,0.990222,0.985803
D19_BANKEN_LOKAL,0.000000,0.000000,0.980360,0.971359,0.980360,0.971359
D19_BANKEN_OFFLINE_DATUM,0.000000,0.000000,0.976534,0.950466,0.976534,0.950466
D19_TELKO_ANZ_12,0.000000,0.000000,0.960388,0.952210,0.960388,0.952210
D19_DIGIT_SERV,0.000000,0.000000,0.959996,0.946053,0.959996,0.946053
D19_BIO_OEKO,0.000000,0.000000,0.955720,0.886222,0.955720,0.886222
D19_TIERARTIKEL,0.000000,0.000000,0.953510,0.947730,0.953510,0.947730
D19_NAHRUNGSERGAENZUNG,0.000000,0.000000,0.953458,0.883320,0.953458,0.883320


In [26]:
# plot the missing an NULL values for each column 
layout = column()

page_size = 32
pages_total = (len(all_data)//page_size)
for k in range(0,len(all_data),page_size):
    plot_data = all_data.iloc[k:k+page_size]

    features = list(plot_data.index.unique())
    cols = ['azdias', 'customers']
    factors = [ (feature, col) for feature in features for col in cols ]

    unk_type = ['null', 'missing']

    null = []
    missing = []
    for feature, source in factors:
        null.append(plot_data.loc[feature, f"{source}_null"])
        missing.append(plot_data.loc[feature, f"{source}_missing"])

    source = ColumnDataSource(data=dict(
        x=factors,
        null=null,
        missing=missing,
    ))

    p = figure(x_range=FactorRange(*factors), plot_width=950, plot_height=500,
               title=f"Overview of missing data in datasets (plot {k//page_size+1} of {pages_total+1})",
               toolbar_location=None, tools="")

    p.vbar_stack(unk_type, x='x', width=0.8, alpha=0.5, color=["blue", "red"], line_color=None, line_width=0, source=source,
                 legend_label=unk_type)

    p.y_range.start = 0
    p.y_range.end = 1.05
    p.xaxis.major_label_orientation = np.pi/2
    p.xaxis.group_label_orientation = np.pi/2
    p.xgrid.grid_line_color = None
    p.legend.location = "top_right"
    p.legend.orientation = "horizontal"

    layout.children.append(p)
show(layout)

### Cleaning ALTER_KINDX columns

In [27]:
# # we can combine these columns to obtain 1 that has fewer missing values... 
# # the "alter" word seems to be used to group into age buckets
# # if we combine them we "only" have 91% of the data missing instead of 99.9%
# len(azdias[['ALTER_KIND1','ALTER_KIND2','ALTER_KIND3','ALTER_KIND4']].mean(axis=1).round().dropna())/len(azdias)

In [28]:
# def muli_columns_row_mean(df, cols, fillna=-1):
#     return df[cols].mean(axis=1).round().fillna(fillna)

# def muli_columns_row_count_non_nan(df, cols):
#     return ((df[cols].isnull() == False).sum(axis=1))

In [29]:
# ALTER_KIND_cols = ['ALTER_KIND1','ALTER_KIND2','ALTER_KIND3','ALTER_KIND4']
# azdias['ALTER_KIND_M'] = muli_columns_row_mean(azdias,ALTER_KIND_cols)
# azdias['ALTER_KIND_C'] = muli_columns_row_count_non_nan(azdias,ALTER_KIND_cols)

# customers['ALTER_KIND_M'] = muli_columns_row_mean(customers,ALTER_KIND_cols)
# customers['ALTER_KIND_C'] = muli_columns_row_count_non_nan(customers,ALTER_KIND_cols)

In [30]:
# azdias.drop(columns=ALTER_KIND_cols, inplace=True)
# customers.drop(columns=ALTER_KIND_cols, inplace=True)

### Cleaning missing data

In [31]:
# if there are more than 1 missing label use the first one, and replace the 2nd one with the first
def clean_missing(df1, missing_labels):
    """
    Makes the missing labels consistent through the dataset. If there is more than 1 missing label, the first one
    is used. Additionally returns which columns in the dataset is missing from the labels, and which lables are
    missing from the dataset.
    
    Input
    df - DataFrame to treat
    missing_labels - a series that has as index the column names and as values a list of the possible missing values
    
    Output
    df_cols_missing_from_labels - columns in the dataset thats is missing from the labels
    label_cols_missing_from_df - columns in the labels that is missing from the dataset
    """
    df_cols_missing_from_labels = list(df1.columns)
    label_cols_missing_from_df = []
    # go through all values that we have a index mapping for
    for c in tqdm(missing_labels.index):
        if c in df_cols_missing_from_labels:
            # remove colum if it has been treated
            df_cols_missing_from_labels.remove(c)
            if len(missing_labels[c]) > 1:
                # if there is more than 1 value to map to, map all of the rest to the first one
                replacement_mapping = dict()
                for k in missing_labels[c][1:]:
                    replacement_mapping[k] = missing_labels[c][0]
                df1[c].replace(replacement_mapping, inplace=True)
        else:
            label_cols_missing_from_df.append(c)
    return df_cols_missing_from_labels, label_cols_missing_from_df

In [32]:
df_cols_missing_from_labels, label_cols_missing_from_df = clean_missing(azdias, missing_labels)
_, _ = clean_missing(customers, missing_labels)

  0%|          | 0/307 [00:00<?, ?it/s]

  0%|          | 0/307 [00:00<?, ?it/s]

In [33]:
azdias.apply(pd.Series.unique).apply(len).sort_values(ascending=False).head(20)

KBA13_ANZAHL_PKW               1262
ANZ_HAUSHALTE_AKTIV             293
GEBURTSJAHR                     117
CAMEO_DEU_2015                   45
LP_LEBENSPHASE_FEIN              42
MIN_GEBAEUDEJAHR                 33
ANZ_PERSONEN                     31
ALTER_HH                         23
ANZ_HH_TITEL                     22
CAMEO_INTL_2015                  22
PRAEGENDE_JUGENDJAHRE            16
LP_LEBENSPHASE_GROB              14
GFK_URLAUBERTYP                  13
LP_FAMILIE_FEIN                  13
D19_GESAMT_ONLINE_QUOTE_12       12
D19_BANKEN_ONLINE_QUOTE_12       12
D19_VERSAND_ONLINE_QUOTE_12      12
ORTSGR_KLS9                      11
LP_STATUS_FEIN                   11
WOHNDAUER_2008                   10
dtype: int64

In [34]:
# t = azdias['ANZ_HAUSHALTE_AKTIV'].copy()
# t.loc[t>16] = 16
# azdias['ANZ_HAUSHALTE_AKTIV'] = t
# azdias['ANZ_HAUSHALTE_AKTIV'].value_counts().sort_index().head(40)/len(azdias)

In [35]:
# t = azdias['ANZ_STATISTISCHE_HAUSHALTE'].copy()
# t.loc[t>16] = 16
# azdias['ANZ_STATISTISCHE_HAUSHALTE'] = t
# azdias['ANZ_STATISTISCHE_HAUSHALTE'].value_counts().sort_index().head(40)/len(azdias)

# Fill missing values

There are various ways to treat missing values. Due to the multiple procesess in which missing data ends up the dataset, it was chosen to not mix these. So if data was lablelled as missing that label was kept (unless there were multiple labels for missing, in which case the first label was used and all other missing labels were replaced with the first). Missing data in the form of NaNs were replaced with the smallest value in the data -1. The data is mostly categorical. 

In [36]:
# replace all nans with a numeric value not found in the data
missing_fill_value = azdias.min()-1
azdias.fillna(missing_fill_value, inplace=True)
customers.fillna(missing_fill_value, inplace=True)

In [37]:
# update the missing labels array so we can use it elsewhere
def merge(r):
    """
    Merges the missing values into a single list. Values in each column does not have to be in a list. Removes NaNs.
    """
    col_vals = list(r)
    arr = []
    for elem in col_vals:
        if '__len__' in dir(elem):  # if __len__ is implemented its a list like objecct
            arr.extend(list(elem))
        elif np.isnan(elem) == False:  # keep it if its not NaN
            arr.append(elem)
    arr = np.array(list(set(arr)))
    arr.sort()
    return arr

missing_labels_all = pd.DataFrame([missing_labels, missing_fill_value]).T
missing_labels_all = missing_labels_all.apply(merge, axis=1)
missing_labels_all

AGER_TYP                [-2.0, -1.0]
ALTERSKATEGORIE_GROB         [-1, 0]
ALTER_HH                 [-1.0, 0.0]
ANREDE_KZ                    [-1, 0]
BALLRAUM                 [-1.0, 0.0]
                            ...     
ANZ_PERSONEN                  [-1.0]
ANZ_TITEL                     [-1.0]
GEBURTSJAHR                   [-1.0]
KBA13_ANZAHL_PKW              [-1.0]
MIN_GEBAEUDEJAHR            [1984.0]
Length: 314, dtype: object

In [38]:
# make sure the operation completed and that there are no longer any nans in the data
azdias.isna().sum().sum() == 0, customers.isna().sum().sum() == 0

(True, True)

# Convert columns to int64 if they dont lose information in the process

In [39]:
can_convert_to_int = []
cannot_convert_to_int = []
for c in azdias.dtypes[azdias.dtypes == np.float64].index:
    # if all values in float form equals the value in int form
    if (azdias[c] == azdias[c].astype(np.int64)).sum() == len(azdias[c]):
        # then we can convert that column to int without losing information
        can_convert_to_int.append(c)
    else:
        # otherwise at last some of the float values contain information that is lost in the conversion
        cannot_convert_to_int.append(c)
        print(f"{c} is not convertable from float to int")
print(f"{len(can_convert_to_int)} columns can be converted from float to int, while {len(cannot_convert_to_int)} can not")        
# convert all to the columns that we can to int
azdias[can_convert_to_int] = azdias[can_convert_to_int].astype(np.int64)
customers[can_convert_to_int] = customers[can_convert_to_int].astype(np.int64)

223 columns can be converted from float to int, while 0 can not


# Find and massage data that has long tails

In [40]:
column_stats = azdias.quantile([0, 0.01, 0.25, 0.5, 0.75, 0.99, 1]).T
column_stats['IQR'] = column_stats[0.75] - column_stats[0.25]
column_stats['l'] = column_stats[0.25] - 1.5 * column_stats['IQR']
column_stats['u'] = column_stats[0.75] + 1.5 * column_stats['IQR']

In [41]:
column_stats['num_vals'] = azdias.apply(pd.Series.unique).apply(len)

In [42]:
column_stats['upper_outlier_percentage'] = (
  #  (azdias < column_stats['l']) | 
    (azdias > column_stats['u'])
).sum()/len(azdias) 

In [43]:
column_stats.head(5)

,0.0,0.01,0.25,0.5,0.75,0.99,1.0,IQR,l,u,num_vals,upper_outlier_percentage
AGER_TYP,-1.0,-1.0,-1.0,-1.0,0.0,3.0,3.0,1.0,-2.5,1.5,5,0.149688
ALTER_HH,-1.0,-1.0,0.0,13.0,17.0,21.0,21.0,17.0,-25.5,42.5,23,0.000000
ANZ_HAUSHALTE_AKTIV,-1.0,-1.0,1.0,3.0,9.0,68.0,595.0,8.0,-11.0,21.0,293,0.069872
ANZ_HH_TITEL,-1.0,-1.0,0.0,0.0,0.0,1.0,23.0,0.0,0.0,0.0,22,0.028538
ANZ_PERSONEN,-1.0,-1.0,1.0,1.0,2.0,5.0,45.0,1.0,-0.5,3.5,31,0.083275


In [44]:
column_stats[(column_stats['num_vals'] > 10) & (column_stats['upper_outlier_percentage'] > 0)]

,0.0,0.01,0.25,0.5,0.75,0.99,1.0,IQR,l,u,num_vals,upper_outlier_percentage
ANZ_HAUSHALTE_AKTIV,-1.0,-1.0,1.0,3.0,9.0,68.0,595.0,8.0,-11.0,21.0,293,0.069872
ANZ_HH_TITEL,-1.0,-1.0,0.0,0.0,0.0,1.0,23.0,0.0,0.0,0.0,22,0.028538
ANZ_PERSONEN,-1.0,-1.0,1.0,1.0,2.0,5.0,45.0,1.0,-0.5,3.5,31,0.083275
D19_BANKEN_ONLINE_QUOTE_12,-1.0,-1.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,12,0.053919
KBA13_ANZAHL_PKW,-1.0,-1.0,346.0,525.0,757.0,1700.0,2300.0,411.0,-270.5,1373.5,1262,0.043839
MIN_GEBAEUDEJAHR,1984.0,1984.0,1992.0,1992.0,1993.0,2009.0,2016.0,1.0,1990.5,1994.5,33,0.132273


In [45]:
# some households with very large number of people
before = azdias['ANZ_PERSONEN'].value_counts().copy()
azdias['ANZ_PERSONEN'].value_counts().sort_index().plot()

Figure(id='2094', ...)

In [46]:
# unlikely that distinguishing between 8 and 9 or even between 8 and 40 will have a significant impact on 
# likelihood of being a customer
# very small % of data points affected with this adjustment
azdias.loc[azdias['ANZ_PERSONEN'] >= 8,'ANZ_PERSONEN'] = 8
customers.loc[customers['ANZ_PERSONEN'] >= 8,'ANZ_PERSONEN'] = 8

In [47]:
after = azdias['ANZ_PERSONEN'].value_counts().copy()

In [48]:
def plot_columns_before_after(before, after, c):
    
    p = figure(plot_width=500, plot_height=400,
               toolbar_location='right', tools="box_zoom")
    
    after = after.sort_index()
    before = before.sort_index()
    
    p.line(before.index, before.values, line_color='blue', legend_label=f'{c} before')
    p.line(after.index,  after.values,  line_color='red',  legend_label=f'{c} after')

    p.title.text = f"{c} transformation"
    
    p.yaxis.axis_label = 'Numer of occurences'
    p.xaxis.axis_label = 'Column values'
    p.xgrid.grid_line_color = None
    return show(p)

plot_columns_before_after(before, azdias['ANZ_PERSONEN'].value_counts().copy(), 'ANZ_PERSONEN')

In [49]:
# The tail of this feature is in steps of 100, while the body is not.
# It does not make sense to have this as a sparse continious variable (could be very prone to overfitting)
# Convert entire feature into buckets of 100
before = azdias['KBA13_ANZAHL_PKW'].value_counts().copy()
azdias['KBA13_ANZAHL_PKW'].value_counts().sort_index().plot()

Figure(id='2537', ...)

In [50]:
# use ceiling for 2 reasons
# 1. having no cars is significantly different to having 1 car
# 2. the distribution has a stop at 1300 when rounding, and with this amount of data that large a step is unlikely to 
#    be natural.  
azdias['KBA13_ANZAHL_PKW'] = (np.ceil((azdias['KBA13_ANZAHL_PKW']/100))*100)
customers['KBA13_ANZAHL_PKW'] = (np.ceil((customers['KBA13_ANZAHL_PKW']/100))*100)

In [51]:
azdias['KBA13_ANZAHL_PKW'].value_counts().sort_index().plot()

Figure(id='2766', ...)

In [52]:
plot_columns_before_after(before, azdias['KBA13_ANZAHL_PKW'].value_counts(), 'KBA13_ANZAHL_PKW')

In [53]:
before = azdias['ANZ_HAUSHALTE_AKTIV'].value_counts()
azdias['ANZ_HAUSHALTE_AKTIV'].value_counts().sort_index().plot()

Figure(id='3275', ...)

In [54]:
# bin data above the median as things are too extreme in the tails
azdias['ANZ_HAUSHALTE_AKTIV'].quantile([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])

0.0    -1.0
0.1     1.0
0.2     1.0
0.3     2.0
0.4     2.0
0.5     3.0
0.6     5.0
0.7     8.0
0.8    11.0
0.9    17.0
Name: ANZ_HAUSHALTE_AKTIV, dtype: float64

In [55]:
ANZ_HAUSHALTE_AKTIV_quantiles = azdias['ANZ_HAUSHALTE_AKTIV'].quantile([0.5,0.6,0.7,0.8,0.9])

In [56]:
ANZ_HAUSHALTE_AKTIV_quantiles.values

array([ 3.,  5.,  8., 11., 17.])

In [57]:
def bin_the_tails(df, column, quantiles):
    """
    Takes a list of values (quantile values) bins all data in between these to the lowest of edges.
    """
    prev = np.inf
    for v in np.flip(quantiles.values):
        df.loc[(df[column] >= v) & (df[column] < prev), column] = v
        prev = v

In [58]:
bin_the_tails(azdias,    'ANZ_HAUSHALTE_AKTIV', ANZ_HAUSHALTE_AKTIV_quantiles)
bin_the_tails(customers, 'ANZ_HAUSHALTE_AKTIV', ANZ_HAUSHALTE_AKTIV_quantiles)

In [59]:
azdias['ANZ_HAUSHALTE_AKTIV'].value_counts().sort_index().plot(title='Distribution after treatment', xlabel='Column values', ylabel='Numer of occurences')

Figure(id='3537', ...)

In [60]:
plot_columns_before_after(before, azdias['ANZ_HAUSHALTE_AKTIV'].value_counts(), 'ANZ_HAUSHALTE_AKTIV')

# Find highly correlated features

In [61]:
# %%time
# a_corr = azdias.corr()

In [62]:
# np.fill_diagonal(a_corr.values, 0)

In [63]:
# pd.DataFrame([a_corr.abs().idxmax(axis=1),a_corr.abs().max(axis=1)], index=['max_corr_col', 'value']).T.sort_values(by='value', ascending=False).head(20)

In [64]:
# a_corr['KBA13_HERST_SONST'].abs().sort_values(ascending=False)

In [65]:
# a_corr['CAMEO_DEU_2015'].abs().sort_values(ascending=False)

In [66]:
correlated_cols_to_drop = [
    'KBA13_FAB_SONSTIGE',   # exact copy of KBA13_HERST_SONST
    'CAMEO_DEUG_2015',      # just a less detailed version of CAMEO_DEU_2015
    'LP_LEBENSPHASE_GROB',  # a GROB suffix is a less detailed version of a FEIN suffix
    'LP_STATUS_GROB',
    'LP_FAMILIE_GROB', 
]

In [67]:
azdias.drop(columns=correlated_cols_to_drop, inplace=True)
customers.drop(columns=correlated_cols_to_drop, inplace=True)

In [68]:
# a_corr_flat = a_corr.stack()

In [69]:
# axis_values = pd.DataFrame(list(a_corr_flat.index), columns=['x','y'])

In [70]:
# a_corr_flat.values.max(), a_corr_flat.values.min()

In [71]:
# factors = list(a_corr.columns)
# x = axis_values['x']
# y = axis_values['y']

# colors = []
# for k in (np.round(a_corr_flat.values*5).astype(int)+5):
#     colors.append(RdYlBu[11][k])

# hm = figure(title="Cross Correlation Heatmap", tools="hover", toolbar_location=None,
#             x_range=factors, y_range=factors, plot_width=950, plot_height=950)

# hm.rect(x, y, color=colors, width=1, height=1)

# hm.xaxis.major_label_orientation = np.pi/2
# hm.yaxis.major_label_text_font_size = "3pt"
# hm.xaxis.major_label_text_font_size = "3pt"

# show(hm)

### missing data discussion

There are two clear ways in which data can be missing. The first is a null in the dataset, caused by a missing value in the original CSV file. The second is data that has explicitly been labelled as unkown or missing in the dataset. It is possible that each of these ways in which data can be missing, could be the result of different mechanisms each containing different information (e.g. the difference between trying to obtain the data and failing, and not trying). It could also be the case that these two different mechanisms are equivalent and thus should be merged to avoid noise in our dataset. 

There are some columns that are mainly missing data - which should be dropped.

There is a clear bias in the datasets between azdias and customers when comparing missing data, most notable in the KBA13 prefixed features, with customers being almost double as likely to have these features missing. Ivestigating and knowing why these fields end up missing would be very important to determine if the fact that the data is missing should be exploited.

In [72]:
# use a KS test to determine if the distributions of the data are similar 
# if the data distribution is too similar then this does not tell us anything about our customers. 

In [73]:
def filter_missing_values(series, missing_vals):
    """
    Returns values that does not represent a value that indicates the value is missing or NaN. 
    
    Input
    Series - The series that should be filtered
    mssing_vals - The list of values that represents a value that is missing.
    
    Output
    Series consisiting only of the values that are not labelled as missing.
    """
    return series[series.isin(missing_vals) == False]

In [74]:
similarity = dict()
for n, c in enumerate(tqdm(azdias.columns)):
    """
    Uses the KS statistic to determine if the distributions are the same.
    If the KS statistic is large or the p-value is small, then we can reject the hypothesis that the distributions 
    of the two samples are the same.
    """
    m = missing_labels_all[c]
#     ks, p = stats.ks_2samp(azdias[c], customers[c])
    ks, p = stats.ks_2samp(filter_missing_values(azdias[c], m),
                           filter_missing_values(customers[c], m))
    similarity[c] = {'ks': ks, 'p': p}
similarity_scores = pd.DataFrame(similarity).T

  0%|          | 0/305 [00:00<?, ?it/s]

In [75]:
similarity_scores.sort_values(by=['ks'], ascending=True).head(25)

,ks,p
SOHO_KZ,0.001451,9.575091e-01
D19_VERSAND_ANZ_12,0.003825,4.027000e-01
KBA13_CCM_1800,0.005202,3.183300e-03
KBA13_MOTOR,0.006814,3.165989e-05
D19_VERSAND_ANZ_24,0.007181,1.457162e-03
KBA05_KRSVAN,0.009083,1.356979e-08
KK_KUNDENTYP,0.009652,1.526343e-05
D19_LEBENSMITTEL,0.011993,2.740967e-02
KBA13_KRSSEG_KLEIN,0.012503,1.391307e-16
D19_BUCH_CD,0.012841,5.709422e-10


In [76]:
def calc_pdf_cdf(data, bins):
    """
    Calculates a PDF and CDF from data using the bins.
    
    Input
    data - list like to calculate PDF and CDF from
    bins - bins to use to group data by 
    
    Output
    bins - bin centers of the PDF and CDF
    pdf - probability distribution of the data
    cdf - cumulative probability distribution of the data
    """
    if np.diff(bins).min() < 1:
        raise Error('minimum bin distance is less than 1')
    else:
        hist_bins = np.append(bins[0] - 0.5, (bins + 0.5))
    count, bins_count = np.histogram(data.loc[np.isfinite(data)], hist_bins)
    pdf = count / sum(count)
    cdf = np.cumsum(pdf)
    return bins, pdf, cdf

In [77]:
def ks_bar_plot(c, azdias=azdias, customers=customers, missing_labels_all=missing_labels_all, attributes=attributes):
    """
    Generates a bar plot for a provided column.
    """
    m = missing_labels_all[c]
    azdi_valid = filter_missing_values(azdias[c], m)
    cust_valid = filter_missing_values(customers[c], m)
    
    numeric_bins = azdi_valid.append(cust_valid).unique()
    numeric_bins.sort()
    
    bins = []
    for i in list(numeric_bins):
        valid_attrib = attributes[(attributes['Attribute'] == c) &
                          (attributes['Value'] == i)]['Meaning']
        if len(valid_attrib) > 0:
            bins.append(valid_attrib.values[0])
        else:
            bins.append(i)
     
#     max_label_len = 0
#     for e in bins:
#         if len(e) > max_label_len:
#             max_label_len = len(e)
    
    x1, pdf1, cdf1 = calc_pdf_cdf(azdi_valid, bins=numeric_bins)
    x2, pdf2, cdf2 = calc_pdf_cdf(cust_valid, bins=numeric_bins)
    
    datasets = ['a', 'c']
    
    x_cat = [ (str(b), d) for b in bins for d in datasets ]
    
    counts_cat = list(pd.DataFrame([pdf1, pdf2], index=datasets).T.stack().values)
    
    source = ColumnDataSource(data=dict(x=x_cat, counts=counts_cat))
    
    p = figure(x_range=FactorRange(*x_cat), plot_width=950, plot_height=500,
               toolbar_location=None, tools="")

#     p.line(x1, pdf1, line_color='blue', legend_label='azdias')
#     p.line(x2, pdf2, line_color='red', legend_label='customer')

    p.vbar(x='x', top='counts', width=0.9, source=source,  line_color=None,
           fill_color=factor_cmap('x', palette=Category10[10][0:2], factors=datasets, start=1, end=2))

    p.title.text = f"{c} [ks:{similarity_scores.loc[c]['ks']:0.4f} p:{similarity_scores.loc[c]['p']:0.4f}]"
    
    p.y_range.start = 0
    p.xaxis.major_label_orientation = 0
    p.xaxis.group_label_orientation = np.pi/2
    p.xgrid.grid_line_color = None
    return p

In [78]:
layout = column()
for c in tqdm(similarity_scores.sort_values(by=['ks'], ascending=True).head(5).index):
    layout.children.append(ks_bar_plot(c))

show(layout)

  0%|          | 0/5 [00:00<?, ?it/s]

In [79]:
similarity_scores[similarity_scores['p'] > 0.001]

,ks,p
D19_DIGIT_SERV,0.021255,0.005409
D19_LEBENSMITTEL,0.011993,0.027410
D19_TELKO_ONLINE_DATUM,0.014892,0.840361
D19_VERSAND_ANZ_12,0.003825,0.402700
D19_VERSAND_ANZ_24,0.007181,0.001457
KBA13_CCM_1800,0.005202,0.003183
SOHO_KZ,0.001451,0.957509
TITEL_KZ,0.030428,0.245682


In [80]:
# Drop columns that are distributed too similarly between datasets that they will not provide separation
ks_cols_to_drop = similarity_scores[similarity_scores['p'] > 0.001].index
azdias.drop(columns=ks_cols_to_drop, inplace=True)
customers.drop(columns=ks_cols_to_drop, inplace=True)

# UMAP

In [81]:
# azdias_c = azdias.copy()
# customers_c = customers.copy()
# azdias_c['customer'] = False
# customers_c['customer'] = True
# combined_data = azdias_c.append(customers_c[azdias_c.columns])

In [82]:
# mini_data = combined_data.sample(100000, random_state=42)
# labels = mini_data['customer']
# train = mini_data.drop(columns=['customer'])

In [83]:
# plots = []
# for n_neighbors in tqdm([200, 500, 1000]):
#     sub_plots = []
#     for min_dist in tqdm([0.3, 0.99]): 
#         # gridsearch revealed default settings perform very well 
#         reducer = umap.UMAP(random_state=42, init='random', n_neighbors=n_neighbors, min_dist=min_dist)
#         reducer.fit(train)
#         embedding = reducer.transform(train)

#         x = embedding[:,0]
#         y = embedding[:,1]
#         colors = labels.replace({False: Cividis[11][-1], True: Cividis[11][0]})

#         p = figure(title=f"Overall view of customers vs non-customers in dataset [n_neighbors = {n_neighbors}, min_dist = {min_dist}]")

#         p.scatter(x, y, radius=0.1,
#                   fill_color=colors, fill_alpha=0.25,
#                   line_color=None)
        
#         sub_plots.append(p)
# #         layout.children.append(p)
#     plots.append(sub_plots)

# grid = gridplot(plots)
# show(grid)

In [84]:
# layout = column()
# for num_cols in [21]: # tqdm(range(15,25,1)):
#     ks_most_dissimilar = similarity_scores.sort_values(by=['ks'], ascending=False).head(num_cols).index
#     ks_mini_data = combined_data[[*ks_most_dissimilar, 'customer']].sample(100000, random_state=42)
#     ks_labels = ks_mini_data['customer']
#     ks_train = ks_mini_data.drop(columns=['customer'])

#     ks_reducer = umap.UMAP(random_state=42, init='random', n_neighbors=15, min_dist=0.1)
#     ks_reducer.fit(ks_train)
#     ks_embedding = ks_reducer.transform(ks_train)

#     x = ks_embedding[:,0]
#     y = ks_embedding[:,1]
#     colors = ks_labels.replace({False: Cividis[11][-1], True: Cividis[11][0]})

#     p = figure(title=f"Overall view of customers vs non-customers in dataset [top {num_cols} dissimilar columns]")

#     p.scatter(x, y, radius=0.1,
#               fill_color=colors, fill_alpha=0.1,
#               line_color=None)
#     layout.children.append(p)

# show(layout)

In [85]:
# %%time
# ks_emb_sample = ks_embedding
# clustering = OPTICS(min_samples=1000, n_jobs=-1)
# clustering.fit(ks_emb_sample)

In [86]:
# max(clustering.labels_)

In [87]:
# cluster_labels = list(pd.DataFrame(clustering.labels_).value_counts().index.get_level_values(0))

In [88]:
# c = 0
# r = dict()
# for l in cluster_labels:
#     if l == -1:
#         r[l] = '#cfcfcf'
#     elif c < len(Category10[10]):
#         r[l] = Category10[10][c]
#         c += 1
#     else:
#         r[l] = '#cfcfcf'

In [89]:
# emb_arr = pd.DataFrame(np.vstack([ks_emb_sample.T, ks_labels.astype(int).values, clustering.labels_, ]).T, columns=['Emb_0', 'Emb_1', 'is_customer', 'Cluster'])
# emb_arr.head(10)

In [90]:
# locations = emb_arr.groupby('Cluster').mean().sort_values(by='is_customer', ascending=False).reset_index()
# locations.merge((emb_arr['Cluster'].value_counts()/len(emb_arr)).rename('Cluster Size'), 
#                 left_on='Cluster', right_index=True)

In [91]:
# colors = list(pd.Series(clustering.labels_).replace(r).values)

# x = ks_embedding[:,0]
# y = ks_embedding[:,1]

# p = figure(title=f"Clusters within the dataset ranked from highest to lowest customer density")

# p.scatter(x, y, radius=0.1,
#           fill_color=colors, fill_alpha=0.25,
#           line_color=None)


# for k in locations.iterrows():
#     l, xl, yl = k[0], k[1]['Emb_0'], k[1]['Emb_1']
#     if l != -1:
#         citation = Label(x=xl, y=yl, # x_units='screen', y_units='screen',
#                          text=str(int(l)), render_mode='css',
#                          border_line_color='black', border_line_alpha=1.0,
#                          background_fill_color='white', background_fill_alpha=1.0)

#         p.add_layout(citation)

# show(p)

In [92]:
# c_cols = list(set(customers.columns) - set(azdias.columns))
# customer_groups = customers[c_cols].copy()

# customer_groups['PRODUCT_GROUP_COSMETIC'] = (customer_groups['PRODUCT_GROUP'].str.find('COSMETIC') >= 0)
# customer_groups['PRODUCT_GROUP_FOOD'] = (customer_groups['PRODUCT_GROUP'].str.find('FOOD') >= 0)
# customer_groups.drop(columns='PRODUCT_GROUP', inplace=True)

In [93]:
# label_groups = pd.merge(labels, customer_groups, left_index=True, right_index=True, how='left')
# label_groups['ONLINE_PURCHASE'].fillna(-1, inplace=True)
# label_groups['CUSTOMER_GROUP'].fillna('NONE', inplace=True)
# label_groups['PRODUCT_GROUP_COSMETIC'].fillna(False, inplace=True)
# label_groups['PRODUCT_GROUP_FOOD'].fillna(False, inplace=True)
# label_groups.head()

In [94]:
# layout = column()
# for k in ['ONLINE_PURCHASE', 'CUSTOMER_GROUP', 'PRODUCT_GROUP_COSMETIC', 'PRODUCT_GROUP_FOOD']:
#     x = embedding[:,0]
#     y = embedding[:,1]
#     if k == 'ONLINE_PURCHASE':
#         colors = label_groups[k].replace({-1: Category20[20][1], 0:Category20[20][4],  1: Category20[20][6]})
#     elif k == 'CUSTOMER_GROUP':
#         colors = label_groups[k].replace({'NONE': Category20[20][1], 'SINGLE_BUYER':Category20[20][4],  'MULTI_BUYER': Category20[20][6]})
#     else: 
#         colors = label_groups[k].replace({False: Category20[20][1], True: Category20[20][6]})
#     p = figure(title=k)

#     p.scatter(x, y, radius=0.1,
#               fill_color=colors, fill_alpha=0.25,
#               line_color=None)

#     layout.children.append(p)
# show(layout)

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

In [95]:
# find features that the distribution of customers and general population differ the most on (ignoring missing values)
most_different = similarity_scores.sort_values(by=['ks'], ascending=False).index

In [96]:
def output_col(c, attributes=attributes):
    p = ks_bar_plot(c)
    show(p)
    display(attributes[attributes['Attribute'] == c])

#### Customers vs  General Population:
- Customers are typically older, growing up in the 40's - 60's, and older than 60 years of age.
    - PRAEGENDE_JUGENDJAHRE
    - ALTERSKATEGORIE_GROB
    - LP_LEBENSPHASE_FEIN
    - ALTER_HH
    - LP_STATUS_FEIN
- Customers are more likely to be male than general population (2:1 vs 1:1)
    - ANREDE_KZ
- Customers are wealthy and have a high income, with a greater chance of being a customer the more wealthy they are. 
    - HH_EINKOMMEN_SCORE
    - LP_STATUS_FEIN
- Customers have a higher interest in investing and saving, and lower in finance minimalism and preparedness
    - FINANZ_SPARER
    - FINANZ_ANLEGER
    - FINANZ_VORSORGER
    - FINANZ_MINIMALIST
- Customers typically own their home
    - LP_LEBENSPHASE_FEIN
    - LP_STATUS_FEIN
    - FINANZ_HAUSBAUER
- Customers live in freestanding homes but more densely populated areas
    - KBA05_ANTG1
    - ANZ_HAUSHALTE_AKTIV
    - KBA05_GBZ
- Customers are more family orientated, and have multiple generations living in the same home
    - LP_FAMILIE_FEIN
    - ANZ_PERSONEN
- Customers are immobile w.r.t. housing, staying at the same location for 10+ years, and not move home
    - WOHNDAUER_2008
    - MOBI_REGIO
- Customers own more cars, and newer cars than the general population
    - KBA05_AUTOQUOT
    - KBA05_VORB2
- Customers are critical, realistic, traditional and religious
    - SEMIO_PFLICHT
    - SEMIO_REL
    - SEMIO_VERT
    - SEMIO_KRIT
    - SEMIO_TRADV
    - SEMIO_LUST
    - SEMIO_RAT


#### Feature analysis
- D19_KONSUMTYP: consumption type
    - Customers are more likely to fall within the universal, versatile and gourmet groups
    - Customers are less likely to fall within the family, informed, modern and inactive groups
- PRAEGENDE_JUGENDJAHRE: dominating movement in the person's youth (avantgarde or mainstream)
    - Customers are more likely to fall in the <= 60ies than > 60ies group
- FINANZ_VORSORGER: financial typology: be prepared
    - Customers rank low and very low on financial preparedness 
    - Customers are less likely to rank average, high or very high on financial preparedness 
- FINANZ_SPARER: financial typology: money saver
    - Customers rank very high on saving money
    - Customers are less likely to rank high, average, low, of very low on saving money
- FINANZ_ANLEGER: financial typology: investor
    - Customers rank very high on investing money
    - Customers are less likely to rank high, average, low, of very low on investing money
- FINANZ_MINIMALIST: financial typology: low financial interest
    - Customers rank very low on financial interest
    - Customers are less likely to rank low, average, high or very high on financial interest
- LP_STATUS_FEIN: social status fine
    - Customers are typically hoseowners, title-holder-households, and top earners
    - Customers are not minimalistic high-income earners, orientationseeking low-income earners, or new houseowners
- SEMIO_RAT: affinity indicating in what way the person is of a rational mind
    - Customers are more likely to be rational individuals
- ALTERSKATEGORIE_GROB: age classification through prename analysis
    - Customer age is typically older than 60 years (this aligns with PRAEGENDE_JUGENDJAHRE)
    - There is allso a large number of customers in the > 45 age group, but this is not larger than the general population
- SEMIO_PFLICHT: affinity indicating in what way the person is dutyfull traditional minded
    - Customers are more likely to show affinity with traditional values
- ZABEOTYP: typification of energy consumers
    - Customers are more likely to identify with green, smart or fair supplied energy types
    - Customers are less likely to identify with price driven, seeking orentation, or indifferent energy types
- HH_EINKOMMEN_SCORE: estimated household net income
    - Customers are of above average to high household income
- ANZ_HAUSHALTE_AKTIV: number of households in the building
    - Customers typically have fewer households in the building than non customers
- SEMIO_LUST: affinity indicating in what way the person is sensual minded
    - Customers have low affinity with sensuality
- LP_LEBENSPHASE_FEIN: lifestage fine
    - Customers are top earners, homeowners, older age
- KBA05_BAUMAX: most common building-type within the cell
    - Customers are located in less densly populated areas with only 1-2 homes in a microcell
- FINANZ_UNAUFFAELLIGER: financial typology: unremarkable
    - HMMM?
- SEMIO_TRADV: affinity indicating in what way the person is traditional minded
    - More towards high affinity 
- SEMIO_KRIT: affinity indicating in what way the person is critical minded
    - Higher affinity with being critically minded
- ALTER_HH: main age within the household
    - Customers birth years are more likely to be < 1955 than >= 1955
- KBA05_ANTG1: number of 1-2 family houses in the cell
    - Homes are typically 1-2 family homes
- ANZ_PERSONEN: number of adult persons in the household
    - More likely to have a larger number of adults in the household 
- LP_FAMILIE_FEIN: familytyp fine
    - Less likely to be single
    - More likely to be a coupe with multi genrations living in the same household
- MOBI_REGIO: moving patterns
    - Low to very low mobility compared to the general population 
- WOHNDAUER_2008: length of residence
    - Living in the same location for more than 10 years
- ANREDE_KZ
    - More likely to be male compared to general population
- SEMIO_VERT: affinity indicating in what way the person is dreamily
    - low afinity to dreaming, i.e. more focused on reality
- SEMIO_REL: affinity indicating in what way the person is religious
    - More affinity with religion
- KBA05_AUTOQUOT: share of cars per household
    - Customers are more likely to have a high number of cars
- KBA05_GBZ: number of buildings in the microcell
    - more likely to have a large number of buildings in the microcell 
- KBA05_VORB2: share of cars with more than two preowner
    - More likely to own newer cars than pre-owned cars
- FINANZ_HAUSBAUER: financial typology: main focus is the own house
    - High affinity with owning a home 

In [97]:
for k in range(50):
    output_col(most_different[k])
    print('='*117*2)

,Attribute,Value,Meaning
384,D19_KONSUMTYP,1,Universal
385,D19_KONSUMTYP,2,Versatile
386,D19_KONSUMTYP,3,Gourmet
387,D19_KONSUMTYP,4,Family
388,D19_KONSUMTYP,5,Informed
389,D19_KONSUMTYP,6,Modern
390,D19_KONSUMTYP,9,Inactive


,Attribute,Value,Meaning
2054,PRAEGENDE_JUGENDJAHRE,"-1, 0",unknown
2055,PRAEGENDE_JUGENDJAHRE,1,"40ies - war years (Mainstream, O+W)"
2056,PRAEGENDE_JUGENDJAHRE,2,"40ies - reconstruction years (Avantgarde, O+W)"
2057,PRAEGENDE_JUGENDJAHRE,3,"50ies - economic miracle (Mainstream, O+W)"
2058,PRAEGENDE_JUGENDJAHRE,4,50ies - milk bar / Individualisation (Avantgar...
2059,PRAEGENDE_JUGENDJAHRE,5,"60ies - economic miracle (Mainstream, O+W)"
2060,PRAEGENDE_JUGENDJAHRE,6,60ies - generation 68 / student protestors (Av...
2061,PRAEGENDE_JUGENDJAHRE,7,60ies - opponents to the building of the Wall ...
2062,PRAEGENDE_JUGENDJAHRE,8,"70ies - family orientation (Mainstream, O+W)"
2063,PRAEGENDE_JUGENDJAHRE,9,"70ies - peace movement (Avantgarde, O+W)"


,Attribute,Value,Meaning
691,FINANZ_VORSORGER,-1,unknown
692,FINANZ_VORSORGER,1,very high
693,FINANZ_VORSORGER,2,high
694,FINANZ_VORSORGER,3,average
695,FINANZ_VORSORGER,4,low
696,FINANZ_VORSORGER,5,very low


,Attribute,Value,Meaning
679,FINANZ_SPARER,-1,unknown
680,FINANZ_SPARER,1,very high
681,FINANZ_SPARER,2,high
682,FINANZ_SPARER,3,average
683,FINANZ_SPARER,4,low
684,FINANZ_SPARER,5,very low


,Attribute,Value,Meaning
661,FINANZ_ANLEGER,-1,unknown
662,FINANZ_ANLEGER,1,very high
663,FINANZ_ANLEGER,2,high
664,FINANZ_ANLEGER,3,average
665,FINANZ_ANLEGER,4,low
666,FINANZ_ANLEGER,5,very low


...

,Attribute,Value,Meaning
138,D19_BANKEN_ANZ_12,0,no transactions known
139,D19_BANKEN_ANZ_12,1,very low activity
140,D19_BANKEN_ANZ_12,2,low activity
141,D19_BANKEN_ANZ_12,3,slightly increased activity
142,D19_BANKEN_ANZ_12,4,increased activity
143,D19_BANKEN_ANZ_12,5,high activity
144,D19_BANKEN_ANZ_12,6,very high activity


,Attribute,Value,Meaning
911,KBA05_KRSAQUOT,"-1, 9",unknown
912,KBA05_KRSAQUOT,1,way below average
913,KBA05_KRSAQUOT,2,below average
914,KBA05_KRSAQUOT,3,average
915,KBA05_KRSAQUOT,4,above average
916,KBA05_KRSAQUOT,5,way above average


,Attribute,Value,Meaning
783,KBA05_ALTER4,"-1, 9",unknown
784,KBA05_ALTER4,0,none
785,KBA05_ALTER4,1,very low
786,KBA05_ALTER4,2,low
787,KBA05_ALTER4,3,average
788,KBA05_ALTER4,4,high
789,KBA05_ALTER4,5,very high


,Attribute,Value,Meaning
1782,KBA13_SEG_SPORTWAGEN,-1,unknown
1783,KBA13_SEG_SPORTWAGEN,0,none
1784,KBA13_SEG_SPORTWAGEN,1,very low
1785,KBA13_SEG_SPORTWAGEN,2,low
1786,KBA13_SEG_SPORTWAGEN,3,average
1787,KBA13_SEG_SPORTWAGEN,4,high
1788,KBA13_SEG_SPORTWAGEN,5,very high


,Attribute,Value,Meaning
991,KBA05_MAXVORB,"-1, 9",unknown
992,KBA05_MAXVORB,1,no preowner
993,KBA05_MAXVORB,2,1 preowner
994,KBA05_MAXVORB,3,2 or more preowner


## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [98]:
from sys import getsizeof

d = dict()
keys = list(locals().keys())
for k in keys:
    d[k] = getsizeof(locals()[k])/1000/1000 # convert to MB

In [99]:
pd.DataFrame(d.values(), columns=['mem size'], index=d.keys()).sort_values(by='mem size', ascending=False).head(20)

,mem size
azdias,1999.980528
customers,379.981394
a_dups,41.837621
c_dups,5.951996
attributes_filt,0.425551
attributes,0.409354
_37,0.070819
missing_labels_all,0.070819
missing_labels,0.061597
column_stats,0.051735


In [100]:
import gc

del azdias
del customers
del a_dups
del c_dups
gc.collect()

172

In [101]:
def apply_pre_processing(df, 
                         low_to_no_info_columns, 
                         correlated_cols_to_drop,
                         ks_cols_to_drop,
                         col_to_proc, 
                         missing_labels, 
                         missing_fill_value,
                         drop_duplicates=True, 
                         replacements=None,
                         ANZ_HAUSHALTE_AKTIV_quantiles=ANZ_HAUSHALTE_AKTIV_quantiles):
    """
    Systematically apply the pre-processing steps uncovered through the analysis process.
    The process 
        1. Removes meta data and sets the index
        2. Drops columns of low value
        3. Drops duplicated rows if asked to do so
        4. Converts columns to numeric equivalents
        5. Unifies the missing labels
        6. Fill NaNs with a unique label (separate from missing labels)
        7. Converts all columns to int
        8. Cleans columns that are prone to ouliers 
        
    Inputs
    df - DataFrame with the data to process
    low_to_no_info_columns - columns identified as low or no info in the analysis process 
    correlated_cols_to_drop - columns identified as highly correlated in the analysis process
    ks_cols_to_drop - columns identified as similarly distributed in the analysis process
    col_to_proc - columns and processes mapping to convert object columns to numeric 
    missing_labels - labels for each column which indicates the missing values 
    missing_fill_value - labels for each column to use for NaN values
    drop_duplicates=True - flag that drops dupliacte rows from the dataset 
    replacements=None - replacements mapping (generated in this function to be re-used for subsequent datasets)
    ANZ_HAUSHALTE_AKTIV_quantiles - bins to use for this column as determined by the analysis process
    
    Output
    The dataframe is processed in place, so its not retuned
    replacements - mapping as calculated in the application of this function
    """
    
    setup_dataframe(df)
    columns_to_drop = [
        *low_to_no_info_columns,  # columns we dont have descriptions for in the accompanying data descriptions
        *correlated_cols_to_drop, # columns that are highly correlated    
        *ks_cols_to_drop,         # columns with the same distribution in customers and general population
    ]
    df.drop(columns=columns_to_drop, inplace=True) 

    # Check for duplicated rows
    if drop_duplicates:
        dups = df.duplicated()
        print(f"Found and dropping {dups.sum()} duplicate rows")
        df.drop(index=(dups[dups==True].index), inplace=True)
    # process non numeric columns so they are numeric
    
    if 'CAMEO_DEUG_2015' in col_to_proc.keys():
        col_to_proc.pop('CAMEO_DEUG_2015')
    apply_cleaning_process(df, col_to_proc)
    
    # convert categoricals to numerics
    if replacements is None:
        replacements = dict()
        cols_to_replace = ['CAMEO_DEU_2015', 'OST_WEST_KZ']
        for c in cols_to_replace:
            replacements[c] = categorical_to_numeric(df, c)
    else:
        for c, r in replacements.items():
            categorical_to_numeric(df, c, r)
    
    # some columns have more than 1 missing label, unify these in the data
    clean_missing(df, missing_labels)
    
    # replace all nans with a numeric value not found in the data
    df.fillna(missing_fill_value, inplace=True)
    
    # analysis showed all columns thus far can be converted to int
    df.astype(np.int64, copy=False)
    
    # apply treatment to some features to reduce outliers or reduce the number of categories
    df.loc[df['ANZ_PERSONEN'] >= 8,'ANZ_PERSONEN'] = 8
    df['KBA13_ANZAHL_PKW'] = (np.ceil((df['KBA13_ANZAHL_PKW']/100))*100)
    bin_the_tails(df, 'ANZ_HAUSHALTE_AKTIV', ANZ_HAUSHALTE_AKTIV_quantiles)
    
    return replacements

In [102]:
train_azdias = load_data('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', 'azdias')
train_customers = load_data('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', 'customers')

azdias parquet found - loading
azdias parquet loaded
customers parquet found - loading
customers parquet loaded


In [103]:
train_customers.drop(columns=['CUSTOMER_GROUP', 'PRODUCT_GROUP', 'ONLINE_PURCHASE'], inplace=True)

In [104]:
replacements = apply_pre_processing(train_azdias, 
                                    low_to_no_info_columns, 
                                    correlated_cols_to_drop,
                                    ks_cols_to_drop,
                                    col_to_proc, 
                                    missing_labels, 
                                    missing_fill_value,
                                    drop_duplicates=True, 
                                    replacements=None)

Found and dropping 52310 duplicate rows


  0%|          | 0/307 [00:00<?, ?it/s]

In [105]:
_ = apply_pre_processing(train_customers, 
                         low_to_no_info_columns, 
                         correlated_cols_to_drop,
                         ks_cols_to_drop,
                         col_to_proc, 
                         missing_labels, 
                         missing_fill_value,
                         drop_duplicates=True, 
                         replacements=replacements)

Found and dropping 42298 duplicate rows


  0%|          | 0/307 [00:00<?, ?it/s]

In [106]:
train_azdias['RESPONSE'] = 0
train_customers['RESPONSE'] = 1

In [107]:
data_train = pd.concat([train_azdias, train_customers], axis=0)

In [108]:
len(train_azdias), len(train_customers), len(data_train)

(838911, 149354, 988265)

In [109]:
mailout_veri = load_data('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', 'mailout_train')

mailout_train parquet found - loading
mailout_train parquet loaded


In [110]:
_ = apply_pre_processing(mailout_veri, 
                         low_to_no_info_columns, 
                         correlated_cols_to_drop,
                         ks_cols_to_drop,
                         col_to_proc, 
                         missing_labels, 
                         missing_fill_value,
                         drop_duplicates=False, 
                         replacements=replacements)

  0%|          | 0/307 [00:00<?, ?it/s]

In [111]:
mailout_veri.head()

,AGER_TYP,ALTER_HH,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_PERSONEN,ANZ_TITEL,BALLRAUM,CAMEO_DEU_2015,CAMEO_INTL_2015,CJT_GESAMTTYP,...,SEMIO_VERT,SHOPPER_TYP,VERS_TYP,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,RESPONSE,ANREDE_KZ,ALTERSKATEGORIE_GROB
LNR,,,,,,,,,,,,,,,,,,,,,
1763,2,8.0,11.0,0.0,1.0,0.0,5.0,22.0,34.0,2.0,...,3,3,2,6.0,9.0,3.0,3,0,2,4
1771,1,13.0,1.0,0.0,2.0,0.0,5.0,20.0,32.0,2.0,...,4,2,1,4.0,9.0,7.0,1,0,2,3
1776,1,9.0,0.0,-1.0,0.0,0.0,1.0,9.0,14.0,4.0,...,7,3,1,-2.0,9.0,2.0,3,0,1,4
1460,2,6.0,3.0,0.0,2.0,0.0,2.0,9.0,14.0,2.0,...,2,1,2,6.0,9.0,1.0,3,0,2,4
1783,2,9.0,17.0,0.0,1.0,0.0,4.0,32.0,41.0,6.0,...,7,1,1,6.0,9.0,3.0,3,0,1,3


In [112]:
X = data_train.drop(columns=['RESPONSE'])
y = data_train['RESPONSE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [113]:
del data_train
del train_azdias
del train_customers
gc.collect()

60

In [114]:
X_valid = mailout_veri.drop(columns=['RESPONSE'])
y_valid = mailout_veri['RESPONSE']

In [115]:
del mailout_veri
gc.collect()

40

In [116]:
y.value_counts()/len(y)

0    0.848873
1    0.151127
Name: RESPONSE, dtype: float64

In [117]:
# %%time
# clf = RandomForestClassifier(random_state=42, n_jobs=-1)
# clf.fit(X_train, y_train)

In [118]:
# y_pred_test = clf.predict(X_test)
# y_score_test = clf.predict_proba(X_test)[:,1]

In [119]:
# y_pred_valid = clf.predict(X_valid)
# y_score_valid = clf.predict_proba(X_valid)[:,1]

In [120]:
# fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_score_test)
# roc_auc_test = auc(fpr_test, tpr_test)

# fpr_valid, tpr_valid, thresholds_valid = roc_curve(y_valid, y_score_valid)
# roc_auc_valid = auc(fpr_valid, tpr_valid)

In [121]:
# p = figure(title="ROC")

# p.line(fpr_test, tpr_test, color='Red', legend_label=f"Test ROC curve (AUC = {roc_auc_test:0.3f})")
# p.line(fpr_valid, tpr_valid, color='Blue', legend_label=f"Validation ROC curve (AUC = {roc_auc_valid:0.3f})")
# p.line([0,1], [0,1], color='Gray')

# p.yaxis.axis_label = "True Positive Rate"
# p.xaxis.axis_label = "False Positive Rate"
# p.legend.location = "bottom_right"

# show(p)

In [122]:
# np.mean(np.array([estimator.tree_.max_depth for estimator in clf.estimators_]))

### Gridsearch

In [123]:
# auc = roc_auc_score(y_test, y_pred)
# tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
# auc, tn, fp, fn, tp

In [124]:
# param_grid = {
#     'n_estimators':      [100, 500],
#     'max_depth':         [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10, 15],
#     'min_samples_leaf':  [1, 2, 5, 10],
#     'max_features':      ['sqrt', 'log2'],
#              }

In [125]:
# clf_hyp = RandomForestClassifier(random_state=42, n_jobs=-1)
# clf_rand = RandomizedSearchCV(estimator=clf_hyp, param_distributions=param_grid, n_iter=(60/2),
#                               verbose=5, random_state=42, scoring='roc_auc')

In [126]:
# %%time
# clf_rand.fit(X, y)

In [127]:
# clf_rand.best_params_

In [128]:
# y_pred_test_rnd = clf_rand.predict(X_test)
# y_score_test_rnd = clf_rand.predict_proba(X_test)[:,1]
# y_pred_valid_rnd = clf_rand.predict(X_valid)
# y_score_valid_rnd = clf_rand.predict_proba(X_valid)[:,1]

In [129]:
# fpr_test_rnd, tpr_test_rnd, thresholds_test_rnd = roc_curve(y_test, y_score_test_rnd)
# roc_auc_test_rnd = auc(fpr_test_rnd, tpr_test_rnd)

# fpr_valid_rnd, tpr_valid_rnd, thresholds_valid_rnd = roc_curve(y_valid, y_score_valid_rnd)
# roc_auc_valid_rnd = auc(fpr_valid_rnd, tpr_valid_rnd)

In [130]:
# p = figure(title="ROC improvement")

# p.line(fpr_test, tpr_test, color='#ff9e9e', legend_label=f"Test ROC curve (AUC = {roc_auc_test:0.3f})")
# p.line(fpr_valid, tpr_valid, color='#a3a3ff', legend_label=f"Validation ROC curve (AUC = {roc_auc_valid:0.3f})")
# p.line(fpr_test_rnd, tpr_test_rnd, color='Red', legend_label=f"Optimised Test ROC curve (AUC = {roc_auc_test_rnd:0.3f})")
# p.line(fpr_valid_rnd, tpr_valid_rnd, color='Blue', legend_label=f"Optimised Validation ROC curve (AUC = {roc_auc_valid_rnd:0.3f})")
# p.line([0,1], [0,1], color='Gray')

# p.yaxis.axis_label = "True Positive Rate"
# p.xaxis.axis_label = "False Positive Rate"
# p.legend.location = "bottom_right"

# show(p)

In [131]:
# results_df = pd.DataFrame(clf_rand.cv_results_)[['params', 'mean_test_score', 'std_test_score']].sort_values(by='mean_test_score', ascending=False)

In [132]:
# for k in clf_rand.best_params_.keys():
#     results_df[k] = results_df['params'].apply(lambda x: x[k])
# results_df.drop(columns=['params'], inplace=True)

In [133]:
# results_df

### Varying n_estimators

In [134]:
# performance_data = dict()

In [135]:
# for k in tqdm(np.geomspace(10,500,18, dtype=int)):
#     clf = RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=k)
#     clf.fit(X_train, y_train)
    
#     y_pred_test = clf.predict(X_test)
#     y_score_test = clf.predict_proba(X_test)[:,1]
    
#     y_pred_valid = clf.predict(X_valid)
#     y_score_valid = clf.predict_proba(X_valid)[:,1]
    
#     fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_score_test)
#     roc_auc_test = auc(fpr_test, tpr_test)

#     fpr_valid, tpr_valid, thresholds_valid = roc_curve(y_valid, y_score_valid)
#     roc_auc_valid = auc(fpr_valid, tpr_valid)
    
#     performance_data[k] = {
#         'y_score_test': y_score_test,
#         'y_score_valid': y_score_valid,
#         'fpr_test': fpr_test, 
#         'tpr_test': tpr_test, 
#         'thresholds_test': thresholds_test,
#         'roc_auc_test': roc_auc_test,
#         'fpr_valid': fpr_valid, 
#         'tpr_valid': tpr_valid, 
#         'thresholds_valid': thresholds_valid,
#         'roc_auc_valid': roc_auc_valid,
#     }
#     print(f"{k}, {roc_auc_test:0.3f}, {roc_auc_valid:0.3f}")

In [136]:
# p = figure(title="ROC for varying n_estimators")

# for k, v in performance_data.items():
    
#     fpr_valid, tpr_valid = v['fpr_valid'], v['tpr_valid']
#     if np.abs(k - 100) < 2:
#         color = 'Blue'
#         alpha = 1
#         legend = 'n_estimators = 100'
#     elif np.abs(k - 126) < 5:
#         color = 'Red'
#         alpha = 1
#         legend = 'n_estimators = 126'
#     else:
#         color = 'Black'
#         alpha = 0.15
#         legend = 'n_estimators variations'
    
# #     p.line(fpr_test, tpr_test, color='Red', legend_label=f"Test ROC curve (AUC = {roc_auc_test:0.3f})")
#     p.line(fpr_valid, tpr_valid, color=color, line_alpha=alpha, legend_label=legend)

# p.line([0,1], [0,1], color='Gray')

# p.yaxis.axis_label = "True Positive Rate"
# p.xaxis.axis_label = "False Positive Rate"
# p.legend.location = "bottom_right"

# show(p)

In [137]:
# p = figure(title="AUC scores by n_estimators")

# n_est, auc_test, auc_valid = [], [], []
# for k, v in performance_data.items():
#     n_est.append(k)
#     auc_test.append(v['roc_auc_test'])
#     auc_valid.append(v['roc_auc_valid'])  
    
# p.line(n_est, auc_test, color='Blue', legend_label=f"Test AUC scores")
# p.line(n_est, auc_valid, color='Red', legend_label=f"Validation AUC scores")

# # p.line([0,1], [0,1], color='Gray')

# p.yaxis.axis_label = "AUC"
# p.xaxis.axis_label = "n_estimators"
# p.legend.location = "bottom_right"

# show(p)

### Stacked classifier

In [138]:
from sklearn.ensemble import StackingClassifier

In [139]:
# clf1 = RandomForestClassifier(n_estimators=15, random_state=42, n_jobs=-1)
# clf2 = RandomForestClassifier(n_estimators=31, random_state=42, n_jobs=-1)
# clf3 = RandomForestClassifier(n_estimators=126, random_state=42, n_jobs=-1)
# # clf4 = RandomForestClassifier(n_estimators=397, random_state=42, n_jobs=-1)

In [140]:
# for c in tqdm([clf1, clf2, clf3]):
#     c.fit(X_train, y_train)


In [141]:
def get_scores(clf, X_valid, y_valid):
    y_score_valid = clf.predict_proba(X_valid)[:,1]
    fpr_valid, tpr_valid, thresholds_valid = roc_curve(y_valid, y_score_valid)
    roc_auc_valid = auc(fpr_valid, tpr_valid)
    return fpr_valid, tpr_valid, roc_auc_valid

# fpr_valid1, tpr_valid1, roc_auc_valid1 = get_scores(clf1, X_valid, y_valid)
# fpr_valid2, tpr_valid2, roc_auc_valid2 = get_scores(clf2, X_valid, y_valid)
# fpr_valid3, tpr_valid3, roc_auc_valid3 = get_scores(clf3, X_valid, y_valid)
# # fpr_valid4, tpr_valid4, roc_auc_valid4 = get_scores(clf4, X_valid, y_valid)

In [142]:
# p = figure(title="ROC Random Forests")

# p.line(fpr_valid1, tpr_valid1, color='Red', legend_label=f"n_estimators=15 (AUC = {roc_auc_valid1:0.3f})")
# p.line(fpr_valid2, tpr_valid2, color='Blue', legend_label=f"n_estimators=31 (AUC = {roc_auc_valid2:0.3f})")
# p.line(fpr_valid3, tpr_valid3, color='Green', legend_label=f"n_estimators=126 (AUC = {roc_auc_valid3:0.3f})")
# # p.line(fpr_valid4, tpr_valid4, color='Purple', legend_label=f"n_estimators=397 (AUC = {roc_auc_valid4:0.3f})")
# p.line([0,1], [0,1], color='Gray')

# p.yaxis.axis_label = "True Positive Rate"
# p.xaxis.axis_label = "False Positive Rate"
# p.legend.location = "bottom_right"

# show(p)

In [143]:
# clfs = StackingClassifier(estimators=[
#     ('n15', RandomForestClassifier(n_estimators=15, random_state=42, n_jobs=1)),
#     ('n31', RandomForestClassifier(n_estimators=31, random_state=42, n_jobs=2)),
#     ('n126', RandomForestClassifier(n_estimators=126, random_state=42, n_jobs=5)),
# #     ('n397', RandomForestClassifier(n_estimators=397, random_state=42)),
#                             ]
#                           )

In [144]:
# clfs.fit(X_train, y_train)

In [145]:
# fpr_valids, tpr_valids, roc_auc_valids = get_scores(clfs, X_valid, y_valid)

In [146]:
# roc_auc_valids

In [147]:
# fpr_valid, tpr_valid = [], []
# for c in clfs.estimators_:
#     fpr_v, tpr_v, roc_auc_valids = get_scores(c, X_valid, y_valid)
#     fpr_valid.append(fpr_v)
#     tpr_valid.append(tpr_v)

In [148]:
# p = figure(title="ROC Stacked Random Forests")

# for fpr, tpr in zip(fpr_valid, tpr_valid):
#     p.line(fpr, tpr, color='Blue', alpha=0.25, legend_label=f"Individual Random Forests")
# p.line(fpr_valids, tpr_valids, color='Red',  legend_label=f"Stacked (AUC = {roc_auc_valids:0.3f})")
# p.line([0,1], [0,1], color='Gray')

# p.yaxis.axis_label = "True Positive Rate"
# p.xaxis.axis_label = "False Positive Rate"
# p.legend.location = "bottom_right"

# show(p)

### Balanced vs unbalanced vs mathing validation set balance

In [149]:
# y_train.value_counts()/len(y_train)

In [150]:
# (y_valid.value_counts()/len(y_valid)).to_dict()

In [151]:
# clf_plain = RandomForestClassifier(n_estimators = 126, random_state=42, n_jobs=-1)
# clf_plain.fit(X_train, y_train)
# print('done1')

# clf_bal = RandomForestClassifier(n_estimators = 126, random_state=42, n_jobs=-1, class_weight='balanced')
# clf_bal.fit(X_train, y_train)
# print('done2')

# clf_balsub = RandomForestClassifier(n_estimators = 126, random_state=42, n_jobs=-1, class_weight='balanced_subsample')
# clf_balsub.fit(X_train, y_train)
# print('done3')

# clf_val = RandomForestClassifier(n_estimators = 126, random_state=42, n_jobs=-1, class_weight=(y_valid.value_counts()/len(y_valid)).to_dict())
# clf_val.fit(X_train, y_train)
# print('done4')

In [152]:
# fpr_valid1, tpr_valid1, roc_auc_valid1 = get_scores(clf_plain, X_valid, y_valid)
# fpr_valid2, tpr_valid2, roc_auc_valid2 = get_scores(clf_bal, X_valid, y_valid)
# fpr_valid3, tpr_valid3, roc_auc_valid3 = get_scores(clf_balsub, X_valid, y_valid)
# fpr_valid4, tpr_valid4, roc_auc_valid4 = get_scores(clf_val, X_valid, y_valid)

In [153]:
# p = figure(title="ROC Balancing")

# p.line(fpr_valid1, tpr_valid1, color='Red', legend_label=f"none (AUC = {roc_auc_valid1:0.3f})")
# p.line(fpr_valid2, tpr_valid2, color='Blue', legend_label=f"balanced (AUC = {roc_auc_valid2:0.3f})")
# p.line(fpr_valid3, tpr_valid3, color='Green', legend_label=f"balanced subsample (AUC = {roc_auc_valid3:0.3f})")
# p.line(fpr_valid4, tpr_valid4, color='Purple', legend_label=f"balanced validation (AUC = {roc_auc_valid4:0.3f})")
# p.line([0,1], [0,1], color='Gray')

# p.yaxis.axis_label = "True Positive Rate"
# p.xaxis.axis_label = "False Positive Rate"
# p.legend.location = "bottom_right"

# show(p)

### Reducing number of features

In [166]:
np.geomspace(8,len(X_train.columns),16,dtype = int, endpoint=False)

array([  8,  10,  12,  15,  19,  24,  31,  38,  48,  61,  76,  95, 120,
       150, 189, 236])

In [157]:
np.flip(np.arange(10,len(X_train.columns),20))

array([290, 270, 250, 230, 210, 190, 170, 150, 130, 110,  90,  70,  50,
        30,  10])

In [167]:
res_dict = dict()
X_lim_t = X_train.copy()
X_lim_v = X_valid.copy()

for rem_cols in tqdm(np.flip(np.geomspace(8,len(X_train.columns),16,dtype = int, endpoint=False))):
    clf = RandomForestClassifier(n_estimators = 126, random_state=42, n_jobs=-1)
    clf.fit(X_lim_t, y_train)
    fpr_valid, tpr_valid, roc_auc_valid = get_scores(clf, X_lim_v, y_valid)
    n = len(X_lim_t.columns)
    res_dict[n] = {
        "score": roc_auc_valid,
        "remaining columns": X_lim_t.columns
    }
    
    print(n, roc_auc_valid)
    step_size = n - rem_cols
    cols_to_drop = pd.DataFrame(clf.feature_importances_, index=X_lim_t.columns).sort_values(by=0).head(step_size).index
    X_lim_t.drop(columns=cols_to_drop, inplace=True)
    X_lim_v.drop(columns=cols_to_drop, inplace=True)

  0%|          | 0/16 [00:00<?, ?it/s]

297 0.8000621102603316
236 0.7982615993790746
189 0.8034525241928767
150 0.799667829720424
120 0.8012495813538089
95 0.7991226593469296
76 0.7978372604856473
61 0.7965670790811581
48 0.8046477701441915
38 0.799730161486677
31 0.8033511409326994
24 0.7968436956756728
19 0.7778185520955346
15 0.773619309291376
12 0.7668181471827105
10 0.7633216762150487


In [180]:
# pd.DataFrame(res_dict).T.sort_index().loc[31]['remaining columns'].to_list()
most_important_columns = ['AGER_TYP',
 'ALTER_HH',
 'CAMEO_DEU_2015',
 'CAMEO_INTL_2015',
 'CJT_GESAMTTYP',
 'D19_GESAMT_DATUM',
 'D19_GESAMT_OFFLINE_DATUM',
 'D19_KONSUMTYP',
 'D19_LOTTO',
 'D19_SONSTIGE',
 'D19_VERSAND_DATUM',
 'D19_VERSAND_OFFLINE_DATUM',
 'FINANZ_ANLEGER',
 'FINANZ_MINIMALIST',
 'FINANZ_SPARER',
 'FINANZ_VORSORGER',
 'GEBURTSJAHR',
 'GFK_URLAUBERTYP',
 'HH_EINKOMMEN_SCORE',
 'INNENSTADT',
 'KBA05_HERST1',
 'KBA05_ZUL4',
 'KBA13_ANZAHL_PKW',
 'KBA13_SEG_SPORTWAGEN',
 'KONSUMNAEHE',
 'LP_LEBENSPHASE_FEIN',
 'LP_STATUS_FEIN',
 'ORTSGR_KLS9',
 'PRAEGENDE_JUGENDJAHRE',
 'REGIOTYP',
 'SEMIO_PFLICHT']

In [173]:
p = figure(title="AUC by number of most important features")

p.line(pd.DataFrame(res_dict).T.index, pd.DataFrame(res_dict).T['score'].values, color='Blue', legend_label=f"AUC")

p.yaxis.axis_label = "AUC"
p.xaxis.axis_label = "number of most important features"
p.legend.location = "bottom_right"

show(p)

### Predict missing values

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = load_data('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', 'mailout_test')

In [ ]:
_ = apply_pre_processing(mailout_test, 
                         low_to_no_info_columns, 
                         correlated_cols_to_drop,
                         ks_cols_to_drop,
                         col_to_proc, 
                         missing_labels, 
                         missing_fill_value,
                         drop_duplicates=False, 
                         replacements=replacements)

In [ ]:
y_score_kaggle = clf.predict_proba(mailout_test)[:,1]

In [ ]:
kaggle = pd.DataFrame(y_score_kaggle, columns=['RESPONSE'], index=mailout_test.index)
kaggle.to_csv('kaggle_valid_mix.csv')

In [ ]:
kaggle.head()